In [94]:
import bs4
from bs4 import BeautifulSoup as soup

# Needed for dynamic web pages
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import urllib

from splinter import Browser

import pandas as pd

In [95]:
# Instantiate an Options object and add the --headless argument
# This runs the browser in the background
opts = Options()
opts.add_argument(" --headless")

# If necessary set the path to your browser's location
# opts.binary_location = os.getcwd() + '\\GoogleChromePortable\GoogleChromePortable.exe'

# Set the location of the webdriver
# chrome_driver = os.getcwd() + '\\chromedriver.exe'
chrome_driver='C:\\Users\\srobi\\Projects\\chromedriver.exe'

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts,executable_path=chrome_driver)

# Load the HTML page from your local drive (if downloaded locally)
# driver.get(os.getcwd() + "\\test.html")

# To scrape a url rather than a local file use this
my_url = 'https://redplanetscience.com/'
driver.get(my_url)

# Put the page source into a variable and create a BS object from it, parse HTML
page_html = driver.page_source
page_soup = soup(page_html, "html.parser")

# Load and print the title and the text of the <div>
print(page_soup.title.get_text())
# print(page_soup.find(id='text').get_text())

# Close the browser
driver.quit()

News - Mars Exploration Program


In [96]:
# Tests to see if html has been parsed properly
print(page_soup.h1.text) # shows header information
print(page_soup.p.text) # shows paragraph information
print(page_soup.body.span.text)
print(page_soup.head.title.text) 

News
NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
MARS Planet Science
News - Mars Exploration Program


In [97]:
print(page_soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [98]:
# Grabs each product
titles = page_soup.findAll("div",{"class":"content_title"})
print(len(titles)) # find total number of containers
news_title = titles[0] # look at the first container
print(news_title.text)

paragraphs = page_soup.findAll("div",{"class":"article_teaser_body"})
print(len(paragraphs)) # find total number of containers
news_p = paragraphs[0] # look at the first container
print(news_p.text)

# title_container = container.findAll("a", {"class":"item_title"})
# title_container

# shipping_container = container.findAll("li", {"class":"price-ship"})
# shipping_container[0].text.strip() # strip removes all white space before or after, new lines, etc

15
NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet
15
The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. 


In [99]:
# Instantiate an Options object and add the --headless argument
# This runs the browser in the background
opts = Options()
opts.add_argument(" --headless")

# If necessary set the path to your browser's location
# opts.binary_location = os.getcwd() + '\\GoogleChromePortable\GoogleChromePortable.exe'

# Set the location of the webdriver
# chrome_driver = os.getcwd() + '\\chromedriver.exe'
chrome_driver='C:\\Users\\srobi\\Projects\\chromedriver.exe'

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts,executable_path=chrome_driver)

# Load the HTML page from your local drive (if downloaded locally)
# driver.get(os.getcwd() + "\\test.html")

# To scrape a url rather than a local file use this
mars_image_url = 'https://spaceimages-mars.com/'
driver.get(mars_image_url)

# Put the page source into a variable and create a BS object from it, parse HTML
page_html = driver.page_source
page_soup = soup(page_html, "html.parser")

# Load and print the title and the text of the <div>
print(page_soup.title.get_text())
# print(page_soup.find(id='text').get_text())

# Close the browser
driver.quit()

# r = requests.get(mars_image_url)
# content = r.content
# soup = BeautifulSoup(content, "html.parser")

# images = soup.findAll('img')[0]
# print(images)

Space Image


In [100]:
# Display featured url
images = page_soup.findAll('img')[1]
featured_image_url = mars_image_url + images['src']
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [101]:
# Pandas scraping
url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(url,header=0)
df = tables[0]
df

[  Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
    Equatorial Diameter:                       6,792 km
 0       Polar Diameter:                       6,752 km
 1                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 2                Moons:          2 ( Phobos & Deimos )
 3       Orbit Distance:       227,943,824 km (1.38 AU)
 4         Orbit Period:           687 days (1.9 years)
 5  Surface Temperature:                   -87 to -5 °C
 6         First Record:              2nd millennium BC
 7          Recorded By:           Egyptian astronomers]

In [103]:
# Convert to HTML table
html_table = df.to_html()
html_table = html_table.replace('\n','')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>1</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>2</th>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>3</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>4</th>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>5</th>      <td>Temperature:</td>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [156]:
# url for the image (Chrome version)
mars_hemispheres_url = 'https://marshemispheres.com/'

# Set up browser for Splinter
executable_path = {'executable_path':'../chromedriver'}
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") # start with window maximized
options.add_argument("--disable-notifications") # notifications turned off
browser = Browser('chrome', **executable_path, headless=False, options = options)

browser.visit(mars_hemispheres_url)

In [157]:
submit = browser.find_by_css('a[class="itemLink product-item"]')
print(f'There are {len(submit)} items in the submit variable.')
# for i in submit:
#     print(i.text)

There are 9 items in the submit variable.


In [158]:
# submit[1].text

In [161]:
# Need to set up a loop to retrieve only even numbers
# Can achieve this by adding 1 and divide by 2, mod needs to == 0
dictionary = {}

for x in range(len(submit)):
    if (x+1) % 2 == 0:
        # Splinter clicks on the submit element to open the page in Chrome
        submit[x].click()

        # Grab the current url using Splinter
        # This will be used as input for Beautiful Soup to parse the HTML
        current_url = browser.url
        print(current_url)
        
        # Instantiate a webdriver
        driver = webdriver.Chrome(options=opts,executable_path=chrome_driver)
        driver.get(current_url)

        # Put the page source into a variable and create a BS object from it, parse HTML
        page_html = driver.page_source
        page_soup = soup(page_html, "html.parser")

        # Load and print the title as a test
        print(page_soup.title.get_text())

        # Close the browser
        driver.quit()

        # Find title of image
        hemisphere_titles = page_soup.find_all("h2", {"class":"title"})
        title = hemisphere_titles[0].text
        print(title)

        # Find image_url
        hemisphere_images = page_soup.find_all("img", {"class":"wide-image"})
        image_url = mars_hemispheres_url + hemisphere_images[0]['src']
        print(image_url)
        
        # Append title and image_url to dictionary
        dictionary[title] = image_url
        
        # Splinter clicks on the browser Back button
        browser.back()

https://marshemispheres.com/cerberus.html
Astropedia Search Results | GUSS Astrogeology Science Center
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=94.0.4606.71)


In [122]:
# To get an overview of the page_soup
# print(page_soup.prettify())

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | GUSS Astrogeology Science Center
  </title>
  <meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <link href="css/main.css" media="screen" rel="stylesheet"/>
  <link href="css/print.css" media="print" rel="stylesheet"/>
  <link href="#" rel="icon" type="image/x-ico"/>
 </head>
 <body id="results">
  <header>
   <a href="#" style="float:right;margin-top:10px;" target="_blank">
    <img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_